# 数据探索

In [1]:
import zipfile

# 默认模式r,读
train_zip = zipfile.ZipFile('../data/Antai_AE_round1_train_20190626.zip')
item_zip = zipfile.ZipFile('../data/Antai_AE_round1_item_attr_20190626.zip') 



In [2]:
print(train_zip.namelist())
print(item_zip.namelist())

['Antai_AE_round1_train_20190626.csv']
['Antai_AE_round1_item_attr_20190626.csv']


In [3]:
train = train_zip.open('Antai_AE_round1_train_20190626.csv')
item = item_zip.open('Antai_AE_round1_item_attr_20190626.csv')

In [4]:
import pandas as pd

train_df = pd.read_csv(train)
train_df.head()

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank
0,xx,8362078,1,2018-08-10 23:49:44,12
1,xx,9694304,2,2018-08-03 23:55:07,9
2,yy,101887,3,2018-08-27 08:31:26,3
3,xx,8131786,3,2018-08-31 06:00:19,9
4,xx,9778613,5,2018-08-21 06:01:56,14


In [5]:
item_df = pd.read_csv(item)
item_df.head()

,item_id,cate_id,store_id,item_price
0,240607,1495,12239,1
1,285232,1498,12239,1
2,113669,1503,12239,1
3,253601,1422,12239,1
4,246568,1497,12239,1


In [6]:
print(train_df.shape[0],train_df.drop_duplicates().shape[0])
print(item_df.shape[0],item_df.drop_duplicates().shape[0])

12868509 12868509
2832669 2832669


In [7]:
test_df = pd.read_csv('../data/Antai_AE_round1_test_20190626.csv')
print(test_df.shape[0])
test_df.head()

166832


,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank
0,yy,1061132,189045,2018-07-17 07:43:40,18
1,yy,1081430,10713670,2018-07-17 09:43:15,8
2,yy,1918135,10546322,2018-07-17 11:11:34,9
3,yy,103277,5199603,2018-07-17 13:27:44,24
4,yy,103277,2477273,2018-07-17 13:27:44,22


# 数据介绍

## 训练数据(train): 
>###   给出xx国的用户的购买数据和yy国的A部分用户的购买数据。
## 测试数据(test): 
>###   给出yy国的B部分用户的购买数据除掉最后一条。

### 字段含义如下：
buyer_country_id: 买家国家id, 只有'xx'和'yy'两种取值 <br/>

buyer_admin_id: 买家id <br/>

item_id: 商品id <br/>

create_order_time: 订单创建时间 <br/>

irank: 每个买家对应的所有记录按照时间顺序的逆排序 <br/>

## 商品属性表(item_attr): 
>###    数据中共涉及2428226个商品，对于其中大部分商品，我们都会给出该商品的类目id、店铺id以及加密价格，其中价格的加密函数f(x)为一个单调增函数。
### 字段含义如下：
item_id：商品id <br/>
cate_id：类目id <br/>
store_id：店铺id <br/>
item_price：加密价格 <br/>
## 要求选手提交的数据
>关于yy国的B部分用户每个用户的最后一条购买数据的预测Top30

In [8]:
train_xx = train_df[train_df['buyer_country_id']=='xx']
train_yy = train_df[train_df['buyer_country_id']=='yy']
print(train_xx.shape[0],train_yy.shape[0])

10635642 2232867


# 购买最多的前100个

In [9]:
train_yy[['buyer_admin_id','item_id']].shape[0],train_yy[['buyer_admin_id','item_id']].drop_duplicates().shape[0]

(2232867, 1770082)

In [10]:
train_yy_1 = train_yy[['buyer_admin_id','item_id']].drop_duplicates()
train_yy_1 = train_yy_1.groupby(['item_id']).size().reset_index().rename(columns={0:'cnt'}).sort_values(by=['cnt'],ascending=False)
train_yy_1

,item_id,cnt
806655,11291525,768
769287,10788165,710
415417,6025215,625
841100,11717821,525
774865,10868663,524
149590,2191537,436
193694,2850195,431
794743,11140901,425
916034,12626807,414
219469,3227771,403


In [11]:
# 
train_xx = train_xx[train_xx['item_id'].isin(item_df['item_id'])==True]
train_xx.shape[0]

10619631

In [12]:
train_xx.head()

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank
0,xx,8362078,1,2018-08-10 23:49:44,12
1,xx,9694304,2,2018-08-03 23:55:07,9
3,xx,8131786,3,2018-08-31 06:00:19,9
4,xx,9778613,5,2018-08-21 06:01:56,14
5,xx,9643757,6,2018-08-27 07:20:43,5


In [12]:
train_xx['item_id'] = train_xx['item_id'].astype('str')


In [13]:
train_xx_1 = train_xx[['buyer_admin_id','item_id']].groupby(['buyer_admin_id']).agg(lambda x: ' '.join(x.values))
train_xx_1.head()

,item_id
buyer_admin_id,
1,1363 149829 311727 837168 928511 1193327 14322...
29,616854 616854 961527 1366136 1587401 1791039 2...
71,277654 277654 277654 280744 580426 3070006 393...
81,111370 3645675 4401623 7159321 9158518 9493430...
89,83309 4013364 6163539 7847626 8151180 8293320 ...


In [15]:
train_yy['item_id'] = train_yy['item_id'].astype('str')
train_yy_1['item_id'] = train_yy_1['item_id'].astype('str')
train_yy_1_1 = train_yy_1[train_yy_1['cnt']>=5]
train_yy = train_yy[train_yy['item_id'].isin(train_yy_1_1['item_id'])]
train_yy_2 = train_yy[['buyer_admin_id','item_id']].groupby(['buyer_admin_id']).agg(lambda x: ' '.join(x.values))
train_yy_2.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,item_id
buyer_admin_id,
111,8816044 8816044 10665523
119,2719441 7659205 11007197
142,8139992 8345489 9110673 9907121 13015745
160,2689986 9984565
169,5793197 5793197 5793197 6456547 8122258 812225...


In [16]:
# xianglianghua
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit_transform(train_yy_2['item_id'])

<128931x52059 sparse matrix of type '<class 'numpy.int64'>'
	with 631999 stored elements in Compressed Sparse Row format>

In [17]:
train_xx_1_v = vec.transform(train_xx_1['item_id'])
train_xx_1_v

<670628x52059 sparse matrix of type '<class 'numpy.int64'>'
	with 1967560 stored elements in Compressed Sparse Row format>

In [18]:
test_df['item_id'] = test_df['item_id'].astype('str')
test_df_1 = test_df[['buyer_admin_id','item_id']].groupby(['buyer_admin_id']).agg(lambda x: ' '.join(x.values))
test_df_1_v = vec.transform(test_df_1['item_id'])
test_df_1_v

<11398x52059 sparse matrix of type '<class 'numpy.int64'>'
	with 47373 stored elements in Compressed Sparse Row format>

In [19]:
# 计算相似度
test_1 = train_xx_1_v * test_df_1_v[3].T
test_1 = test_1.toarray()
import numpy as np
print(np.where(test_1>0))
test_1[np.where(test_1>0)]

(array([  1013,   1039,  19570,  29651,  29721,  32259,  33010,  35187,
        37140,  43004,  48220,  48546,  52761,  56021,  56780,  58331,
        60136,  62394,  62519,  65125,  65392,  67557,  69606,  69780,
        71127,  71358,  74409,  75279,  80365,  83070,  85581,  87562,
        88854,  90502,  92591,  93302,  95046,  97460, 105609, 106354,
       106566, 114252, 123485, 124210, 127954, 131232, 135045, 135300,
       135946, 138441, 145052, 146755, 150937, 151013, 154986, 156810,
       157273, 159328, 164374, 165018, 165658, 169534, 172368, 174300,
       176504, 179723, 179885, 180411, 181124, 181175, 182764, 183877,
       185501, 185811, 187729, 189885, 190958, 191541, 195355, 199783,
       200504, 203538, 204252, 205227, 206013, 207783, 210340, 211685,
       212262, 213689, 215279, 216029, 223282, 228712, 229818, 229873,
       231164, 231451, 231473, 231637, 233634, 234365, 235460, 238540,
       238755, 245164, 247328, 250025, 258011, 260534, 261185, 262435,
     

array([ 2,  1,  2,  2,  2,  2,  2,  2,  2,  1,  2,  2,  2,  2,  1,  2,  2,
        1,  2,  2,  1,  2,  2,  2,  2,  2,  2,  1,  2,  2,  1,  6,  2,  2,
        1,  6,  2,  2,  4,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  4,  2,
        2,  4,  4,  2,  6,  1,  2,  1,  2,  2,  4,  1,  2,  2,  6,  2, 10,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  6,  2,  2,  2,  2,  1,  2,
        2,  2,  6,  2,  1,  2,  4,  1,  2,  8,  2,  2,  1,  1,  2,  1,  2,
        2,  2,  2,  2,  2,  2,  1,  2,  2,  1,  2,  2,  1,  2,  2,  4,  2,
        1,  2,  2,  2,  1,  1,  2,  2,  2,  2,  2,  2,  1,  2,  2,  2,  2,
        2,  2,  4,  1,  1,  2,  4,  2,  2,  2,  1,  2,  2,  2,  4,  1,  2,
        1,  4,  2,  2,  2,  2,  2,  2,  2,  4,  2,  4,  2,  2,  2,  2,  2,
        2,  2,  2,  4,  2,  6,  2,  2,  1,  2,  2,  1,  2,  2,  4,  4,  2,
        4,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  4,  2,  2,  2,  6,  6,  6,  2,  1,  2,  1,  2,  1,  2,
        2,  2,  2,  2,  2

In [66]:
test_df_1

,item_id
buyer_admin_id,
152,8351840 8472223 9891513 8410857 8064216 401606...
282,6547607 9677940 10808393 11536575 688799 11310...
321,10570664 9759248 10000188 10000188 9759248 263...
809,2347616 2262443 5492003 5707010 6339286 120757...
870,8952725 6787506 6163411 7412272 10234567 43436...
2410,11573165 7667925 1156153 3242952 904577 115731...
2956,7535052 7358377 745619 1284838 7644819 1866761...
3866,1192286 3388420 3991184 1192286 4428949 119228...
7277,6561610 530503 5482394 6775644 12441577 123570...


In [79]:
test_df_1.iloc[0,].values[0]


'8351840 8472223 9891513 8410857 8064216 4016066 7937154'

In [82]:
test_df_1.iloc[0,].name

152

In [90]:
# 关于yy国的B部分用户每个用户的最后一条购买数据的预测Top30
# 请选手提交的CSV文件, 其格式如下：
# buyer_admin_id,predict 1,predict 2,…,predict 30

import numpy as np

result = []
for i in range(test_df_1_v.shape[0]):
    test_i = test_df_1_v[i]
    test_1 = train_xx_1_v * test_i.T
    test_1 = test_1.toarray()
    corr_mat = np.argsort(-test_1.T,axis=1)
#     print(corr_mat)
#     corr_top_lst = corr_mat[:,0:2][0]
#     print(train_xx_1.iloc[corr_top_lst,:].values)
#     print(" ".join(train_xx_1.iloc[corr_top_lst,:].values.flatten()))
    j = 1
    top_n = 0
    top_30 = 30
    item_top = []
    while top_n<top_30:
        corr_top_lst = corr_mat[:,0:j][0]
        item_lst_clue = " ".join(train_xx_1.iloc[corr_top_lst,:].values.flatten()).split(" ")
        item_lst_have = test_df_1.iloc[i,].values[0].split(" ")
        item_top = set(item_lst_clue) - set(item_lst_have)
        top_n = len(item_top)
        j = j+1
#         print(item_lst_clue)
#         print(item_lst_have)
#         print(top_n)
    result.append([test_df_1.iloc[i,].name] + list(item_top)[0:30])

result = pd.DataFrame(result)
result

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,152,11793723,10923467,8691326,8988335,10858133,12144154,11171347,8522599,8165456,...,11598643,5517808,9890103,322698,1589650,2213546,4998345,7839968,1464587,6379477
1,282,280550,4404600,5993014,7757698,2021623,5999449,2158247,3310757,10685119,...,7913586,11537538,5940155,2086048,6437189,8732749,8927403,3437726,7575596,7444591
2,321,593003,2969216,5748179,5190873,4305422,2289447,2290168,6267352,10822350,...,11235982,756195,109455,872426,1494329,6737851,590493,773172,1472551,324223
3,809,4844691,5378133,4554036,5326024,1444370,1938797,1656268,4218750,8235348,...,9929813,2390373,6774235,8553904,5657461,2194480,9159439,1812208,4564754,741932
4,870,8227677,8495485,46835,9860804,10745176,8371312,8351782,3809865,8473643,...,9427219,3631710,4432324,8599974,5658963,8585210,4916789,4030451,2764880,8584105
5,2410,10039023,11711248,11765282,1469491,10923467,12043751,12891086,5435662,11396001,...,6730824,11851617,12247625,7744706,4087442,11616506,8320859,12509003,4529517,10848192
6,2956,5557256,3200473,1363,1668449,2357621,311727,11433672,11604770,6442985,...,7616663,1193327,6004191,10864714,149829,8248021,2027711,2129161,2010333,9717330
7,3866,8651285,5414011,8471739,1032078,47230,2300770,6194163,3824319,8454107,...,877209,3795310,9639586,5932992,6036366,9393503,8608721,9770150,11572923,889079
8,7277,3307115,8021966,6128784,11747910,11047354,2343979,8558694,2501923,5610461,...,8063066,9274238,5632949,5311330,12134022,8898833,5546673,773110,848996,7782819
9,7944,5283524,4768512,8732449,11700403,5770080,10408292,9773539,12397623,9623963,...,11311485,11218030,9180814,10514346,9581905,11057362,12022768,13040248,8396661,11479003


In [91]:
result.to_csv('../data/submit_1.csv',header=None,index=False)